In [96]:
!pip install pytorch_tabnet
!pip install iterative-stratification

In [97]:
import pandas as pd
import numpy as np
from google.colab import drive
from types import SimpleNamespace
from numpy import isnan
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold

In [98]:
drive.mount('/Content/')

Drive already mounted at /Content/; to attempt to forcibly remount, call drive.mount("/Content/", force_remount=True).


In [99]:
conf = SimpleNamespace()
conf.path = '/Content/MyDrive/DataSets/ODB/'

tabnet_params = dict(n_d=24, n_a=24, n_steps=1, gamma=1.3,
                     device_name='cuda',
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                    #  mask_type='entmax',
                     mask_type="sparsemax",
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )


# verbose=10, device_name='cuda', seed=42,
#                                 scheduler_params=dict(mode="min",
#                                                patience=5,
#                                                min_lr=1e-7,
#                                                factor=0.9,),
#                                 optimizer = AdamW( 
#                                 lr=2e-5, 
#                                 correct_bias=False,
#                                 weight_decay=0.01,
#                                 )
#                                 scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau)

# Подготовка данных

In [100]:
data_df = pd.read_csv(conf.path + 'data.csv')
data_df

,incident,flg_90_12_add,APPLICATION_MONTH,Category_Feature_0,Category_Feature_1,Category_Feature_2,Category_Feature_3,Category_Feature_4,Category_Feature_5,Category_Feature_6,Category_Feature_7,Category_Feature_8,Category_Feature_9,Category_Feature_10,Category_Feature_11,Category_Feature_12,Category_Feature_13,Category_Feature_14,Category_Feature_15,Category_Feature_16,Category_Feature_17,Category_Feature_18,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,...,Feature_1848,Feature_1849,Feature_1850,Feature_1851,Feature_1852,Feature_1853,Feature_1854,Feature_1855,Feature_1856,Feature_1857,Feature_1858,Feature_1859,Feature_1860,Feature_1861,Feature_1862,Feature_1863,Feature_1864,Feature_1865,Feature_1866,Feature_1867,Feature_1868,Feature_1869,Feature_1870,Feature_1871,Feature_1872,Feature_1873,Feature_1874,Feature_1875,Feature_1876,Feature_1877,Feature_1878,Feature_1879,Feature_1880,Feature_1881,Feature_1882,Feature_1883,Feature_1884,Feature_1885,Feature_1886,Feature_1887
0,14511110,0.0,2018-10,6.0,2,25,2.0,0.0,3,1,2.0,35000.0000,0.0,0.0,1.0,0.0,2.0,7.0,4.0,28.0,6.0,3.0,NaN,NaN,NaN,0.225000,0.175000,0,0,1,1,0,0,0,1,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5.053696,5.053696,5.053696,5.053696,5.053696,5.053696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,20184
1,15018800,0.0,2018-12,1.0,2,49,4.0,0.0,3,1,2.0,50000.0000,0.0,0.0,0.0,0.0,2.0,7.0,5.0,23.0,3.0,4.0,NaN,NaN,NaN,0.333333,0.166667,0,0,7,1,0,0,0,4,1,0,0,0,3,...,1.901645,1.0,12969.251635,27353.234548,14383.982913,14383.982913,0.474140,0.901645,0.901645,1.901645,1.901645,1.0,0,0.526316,0.526316,0.600000,0.600000,0.526316,0.526316,0.043231,0.091177,0.047947,0.047947,0.043231,0.091177,0.047947,0.047947,0.129693,0.273532,0.143840,0.143840,0.259385,0.547065,0.287680,0.287680,0,1,1,1,20184
2,14499737,0.0,2018-10,6.0,2,24,1.0,0.0,3,0,1.0,20000.0000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.253165,0.253165,0,1,10,2,2,0,1,10,5,2,0,1,8,...,NaN,NaN,8406.411307,4853.739804,NaN,NaN,1.731945,NaN,NaN,NaN,NaN,NaN,0,1.128571,1.128571,0.790000,0.790000,0.087778,0.087778,0.106410,0.061440,NaN,NaN,0.106410,0.061440,NaN,NaN,0.420321,0.242687,NaN,NaN,0.420321,0.242687,NaN,NaN,0,1,0,1,20184
3,14953427,0.0,2018-12,4.0,2,26,1.0,0.0,3,1,2.0,30000.0000,0.0,0.0,0.0,0.0,2.0,7.0,1.0,32.0,1.0,4.0,NaN,NaN,NaN,0.650000,0.150000,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,1,20184
4,14744453,0.0,2018-11,6.0,2,30,4.0,2.0,3,1,3.0,50000.0000,0.0,0.0,1.0,0.0,2.0,7.0,5.0,22.0,5.0,4.0,NaN,NaN,NaN,0.900000,0.250000,0,1,3,3,0,0,1,3,4,0,0,1,3,...,NaN,NaN,2293.148913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.883465,2.883465,10.473946,10.473946,0.188679,0.188679,0.011466,NaN,NaN,NaN,0.011466,NaN,NaN,NaN,0.012740,NaN,NaN,NaN,0.045863,NaN,NaN,NaN,0,1,0,1,20184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82612,20559842,NaN,2020-01,1.0,2,34,2.0,2.0,3,0,2.0,35000.0000,0.0,0.0,0.0,0.0,0.0,8.0,5.0,22.0,5.0,3.0,3.833333,1.166667,0.129032,1.437500,0.437500,0,0,16,2,4,0,0,5,2,4,0,0,4,...,NaN,NaN,12535.463590,10555.030549,6554.481101,6554.481101,1.187629,1.912503,1.912503,1.610353,1.610353,1.0,0,0.400000,0.400000,1.938219,1.938219,0.400000,0.400000,0.156693,0.131938,0.081931,NaN,0.156693,0.131938,0.081931,0.081931,0.109004,0.091783,0.056995,NaN,0.358156,0.301572,0.187271,NaN,0,0,0,1,20201
82613,20380255,NaN,2020-01,10.0,2,66,2.0,0.0,5,1,3.0,19000.0000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,2.133333,0.633333,0.403226,0.256000,0.07

In [101]:
model_df = pd.read_csv(conf.path + 'model.csv')
model_df

,incident,PD
0,14511110,0.022191
1,15018800,0.019972
2,14499737,0.043884
3,14953427,0.013412
4,14744453,0.033354
...,...,...
82612,20559842,NaN
82613,20380255,NaN
82614,20348581,NaN
82615,21118162,NaN


In [102]:
sample_submission = pd.read_csv(conf.path + 'sample_submission.csv')
sample_submission

,0.5488135039273248
0,0.715189
1,0.602763
2,0.544883
3,0.423655
4,0.645894
...,...
994,0.097676
995,0.514922
996,0.938412
997,0.228647


In [103]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82617 entries, 0 to 82616
Columns: 1910 entries, incident to Feature_1887
dtypes: float64(1837), int64(72), object(1)
memory usage: 1.2+ GB


In [104]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82617 entries, 0 to 82616
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   incident  82617 non-null  int64  
 1   PD        81617 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.3 MB


In [105]:
data_repare = data_df.drop(['incident','APPLICATION_MONTH', 'flg_90_12_add'], axis=1)

In [106]:
# dddd = data_df.drop(['incident','APPLICATION_MONTH', 'flg_90_12_add'], axis=1)
# ccc = 0
# for dd in dddd.eq(-1).values.flatten():
#     if dd:
#         ccc+=1
# ccc

## Вычисляем индекс нужности фич

In [107]:
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name='MI Scores', index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [108]:
data_repare

,Category_Feature_0,Category_Feature_1,Category_Feature_2,Category_Feature_3,Category_Feature_4,Category_Feature_5,Category_Feature_6,Category_Feature_7,Category_Feature_8,Category_Feature_9,Category_Feature_10,Category_Feature_11,Category_Feature_12,Category_Feature_13,Category_Feature_14,Category_Feature_15,Category_Feature_16,Category_Feature_17,Category_Feature_18,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,...,Feature_1848,Feature_1849,Feature_1850,Feature_1851,Feature_1852,Feature_1853,Feature_1854,Feature_1855,Feature_1856,Feature_1857,Feature_1858,Feature_1859,Feature_1860,Feature_1861,Feature_1862,Feature_1863,Feature_1864,Feature_1865,Feature_1866,Feature_1867,Feature_1868,Feature_1869,Feature_1870,Feature_1871,Feature_1872,Feature_1873,Feature_1874,Feature_1875,Feature_1876,Feature_1877,Feature_1878,Feature_1879,Feature_1880,Feature_1881,Feature_1882,Feature_1883,Feature_1884,Feature_1885,Feature_1886,Feature_1887
0,6.0,2,25,2.0,0.0,3,1,2.0,35000.0000,0.0,0.0,1.0,0.0,2.0,7.0,4.0,28.0,6.0,3.0,NaN,NaN,NaN,0.225000,0.175000,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5.053696,5.053696,5.053696,5.053696,5.053696,5.053696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,20184
1,1.0,2,49,4.0,0.0,3,1,2.0,50000.0000,0.0,0.0,0.0,0.0,2.0,7.0,5.0,23.0,3.0,4.0,NaN,NaN,NaN,0.333333,0.166667,0,0,7,1,0,0,0,4,1,0,0,0,3,1,0,0,...,1.901645,1.0,12969.251635,27353.234548,14383.982913,14383.982913,0.474140,0.901645,0.901645,1.901645,1.901645,1.0,0,0.526316,0.526316,0.600000,0.600000,0.526316,0.526316,0.043231,0.091177,0.047947,0.047947,0.043231,0.091177,0.047947,0.047947,0.129693,0.273532,0.143840,0.143840,0.259385,0.547065,0.287680,0.287680,0,1,1,1,20184
2,6.0,2,24,1.0,0.0,3,0,1.0,20000.0000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.253165,0.253165,0,1,10,2,2,0,1,10,5,2,0,1,8,5,2,0,...,NaN,NaN,8406.411307,4853.739804,NaN,NaN,1.731945,NaN,NaN,NaN,NaN,NaN,0,1.128571,1.128571,0.790000,0.790000,0.087778,0.087778,0.106410,0.061440,NaN,NaN,0.106410,0.061440,NaN,NaN,0.420321,0.242687,NaN,NaN,0.420321,0.242687,NaN,NaN,0,1,0,1,20184
3,4.0,2,26,1.0,0.0,3,1,2.0,30000.0000,0.0,0.0,0.0,0.0,2.0,7.0,1.0,32.0,1.0,4.0,NaN,NaN,NaN,0.650000,0.150000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,1,20184
4,6.0,2,30,4.0,2.0,3,1,3.0,50000.0000,0.0,0.0,1.0,0.0,2.0,7.0,5.0,22.0,5.0,4.0,NaN,NaN,NaN,0.900000,0.250000,0,1,3,3,0,0,1,3,4,0,0,1,3,4,0,0,...,NaN,NaN,2293.148913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.883465,2.883465,10.473946,10.473946,0.188679,0.188679,0.011466,NaN,NaN,NaN,0.011466,NaN,NaN,NaN,0.012740,NaN,NaN,NaN,0.045863,NaN,NaN,NaN,0,1,0,1,20184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82612,1.0,2,34,2.0,2.0,3,0,2.0,35000.0000,0.0,0.0,0.0,0.0,0.0,8.0,5.0,22.0,5.0,3.0,3.833333,1.166667,0.129032,1.437500,0.437500,0,0,16,2,4,0,0,5,2,4,0,0,4,2,0,0,...,NaN,NaN,12535.463590,10555.030549,6554.481101,6554.481101,1.187629,1.912503,1.912503,1.610353,1.610353,1.0,0,0.400000,0.400000,1.938219,1.938219,0.400000,0.400000,0.156693,0.131938,0.081931,NaN,0.156693,0.131938,0.081931,0.081931,0.109004,0.091783,0.056995,NaN,0.358156,0.301572,0.187271,NaN,0,0,0,1,20201
82613,10.0,2,66,2.0,0.0,5,1,3.0,19000.0000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,2.133333,0.633333,0.403226,0.256000,0.076000,0,1,4,2,0,0,1,2,2,0,0,0,2,2,0,0,...,1.000000,1.0,7430.360418,7430.360418,7430.360418,7430.360418,1.000000,1.000000

In [109]:
for col_name in data_repare.columns:
    mask = data_repare[col_name].dropna().index
    data_repare.loc[mask, col_name] = (data_repare.loc[mask, col_name] -
                                       data_repare.loc[mask, col_name].min()) / (data_repare.loc[mask, col_name].max() -
                                                                                 data_repare.loc[mask, col_name].min())

In [110]:
data_repare

,Category_Feature_0,Category_Feature_1,Category_Feature_2,Category_Feature_3,Category_Feature_4,Category_Feature_5,Category_Feature_6,Category_Feature_7,Category_Feature_8,Category_Feature_9,Category_Feature_10,Category_Feature_11,Category_Feature_12,Category_Feature_13,Category_Feature_14,Category_Feature_15,Category_Feature_16,Category_Feature_17,Category_Feature_18,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,...,Feature_1848,Feature_1849,Feature_1850,Feature_1851,Feature_1852,Feature_1853,Feature_1854,Feature_1855,Feature_1856,Feature_1857,Feature_1858,Feature_1859,Feature_1860,Feature_1861,Feature_1862,Feature_1863,Feature_1864,Feature_1865,Feature_1866,Feature_1867,Feature_1868,Feature_1869,Feature_1870,Feature_1871,Feature_1872,Feature_1873,Feature_1874,Feature_1875,Feature_1876,Feature_1877,Feature_1878,Feature_1879,Feature_1880,Feature_1881,Feature_1882,Feature_1883,Feature_1884,Feature_1885,Feature_1886,Feature_1887
0,0.357143,1.0,0.074074,0.2,0.000000,0.333333,1.0,0.333333,0.001269,0.0,0.0,1.0,0.0,0.666667,0.538462,0.8,0.87500,1.0,0.272727,NaN,NaN,NaN,0.000018,0.000634,0.0,0.000000,0.005076,0.021739,0.000000,0.0,0.000000,0.006135,0.030303,0.000000,0.0,0.000000,0.006944,0.041667,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000212,0.000029,8.418142e-06,3.367257e-06,3.387547e-07,3.389856e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0
1,0.000000,1.0,0.518519,0.6,0.000000,0.333333,1.0,0.333333,0.001813,0.0,0.0,0.0,0.0,0.666667,0.538462,1.0,0.71875,0.4,0.363636,NaN,NaN,NaN,0.000027,0.000604,0.0,0.000000,0.035533,0.021739,0.000000,0.0,0.000000,0.024540,0.030303,0.000000,0.0,0.000000,0.020833,0.041667,0.000000,0.0,...,0.004527,0.007162,0.002064,0.004380,0.020691,0.024185,0.000469,0.000898,0.001260,0.004419,0.002655,0.006785,0.0,0.000021,0.000003,9.953163e-07,3.981265e-07,3.490374e-08,3.513460e-08,0.004142,0.008762,0.010349,0.016530,0.004149,0.008762,0.010349,0.016530,0.000011,0.000024,0.000013,0.000013,0.000015,0.000050,0.000026,0.000026,0.0,1.0,1.0,1.0,0.0
2,0.357143,1.0,0.055556,0.0,0.000000,0.333333,0.0,0.000000,0.000725,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000020,0.000918,0.0,0.090909,0.050761,0.043478,0.007519,0.0,0.166667,0.061350,0.151515,0.007519,0.0,0.166667,0.055556,0.208333,0.007692,0.0,...,NaN,NaN,0.001332,0.000775,NaN,NaN,0.001728,NaN,NaN,NaN,NaN,NaN,0.0,0.000046,0.000006,1.311983e-06,5.247932e-07,5.471663e-09,5.702522e-09,0.010217,0.005903,NaN,NaN,0.010223,0.005903,NaN,NaN,0.000037,0.000021,NaN,NaN,0.000024,0.000022,NaN,NaN,0.0,1.0,0.0,1.0,0.0
3,0.214286,1.0,0.092593,0.0,0.000000,0.333333,1.0,0.333333,0.001088,0.0,0.0,0.0,0.0,0.666667,0.538462,0.2,1.00000,0.0,0.363636,NaN,NaN,NaN,0.000052,0.000544,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0
4,0.357143,1.0,0.166667,0.6,0.007843,0.333333,1.0,0.666667,0.001813,0.0,0.0,1.0,0.0,0.666667,0.538462,1.0,0.68750,0.8,0.363636,NaN,NaN,NaN,0.000072,0.000906,0.0,0.090909,0.015228,0.065217,0.000000,0.0,0.166667,0.018405,0.121212,0.000000,0.0,0.166667,0.020833,0.166667,0.000000,0.0,...,NaN,NaN,0.000353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000120,0.000016,1.745189e-05,6.980757e-06,1.224357e-08,1.247443e-08,0.001088,NaN,NaN,NaN,0.001095,NaN,NaN,NaN,0.000001,NaN,NaN,NaN,0.000003,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [111]:
data_repare = data_repare.fillna(-1)
data_repare['target'] = np.abs(model_df['PD'] - data_df['flg_90_12_add'])
train = data_repare[~data_repare['target'].isna()]

In [52]:
# mi_scores = make_mi_scores(train[:20000].drop('target', axis=1).reset_index(drop=True),
#                            train[:20000]['target'].reset_index(drop=True))

In [112]:
mi_scores = pd.read_csv(conf.path + 'good_scores.csv')
mi_scores

,Unnamed: 0,index,MI Scores
0,0,Category_Feature_2,1.083351e-01
1,1,Feature_962,9.536386e-02
2,2,Feature_1083,9.268727e-02
3,3,Feature_960,9.235481e-02
4,4,Feature_69,8.621765e-02
...,...,...,...
1868,1868,Feature_675,1.110223e-15
1869,1869,Feature_117,1.110223e-15
1870,1870,Feature_149,1.110223e-15
1871,1871,Feature_707,1.110223e-15


In [113]:
mi_scores = mi_scores.reset_index()
columns_with_scores = mi_scores[mi_scores['MI Scores'] > 0]
columns_with_scores

,level_0,Unnamed: 0,index,MI Scores
0,0,0,Category_Feature_2,1.083351e-01
1,1,1,Feature_962,9.536386e-02
2,2,2,Feature_1083,9.268727e-02
3,3,3,Feature_960,9.235481e-02
4,4,4,Feature_69,8.621765e-02
...,...,...,...,...
1868,1868,1868,Feature_675,1.110223e-15
1869,1869,1869,Feature_117,1.110223e-15
1870,1870,1870,Feature_149,1.110223e-15
1871,1871,1871,Feature_707,1.110223e-15


In [114]:
good_scores = list(columns_with_scores[columns_with_scores['MI Scores'] > 0]['index'].values)

In [56]:
# columns_with_scores.to_csv(conf.path + 'good_scores.csv')

In [57]:
# get_zero_count = data_repare.isnull().sum().reset_index()
# get_zero_count.columns = ['column_name', 'count']
# get_zero_count = get_zero_count[get_zero_count['count'] > 0]
# get_zero_count

In [58]:
# get_zero_count['per_drop'] = 0
# l_data = len(data_repare)
# for cname in get_zero_count['column_name']:
#     count = get_zero_count.loc[get_zero_count['column_name'] == cname, 'count'].values[0]
#     get_zero_count.loc[get_zero_count['column_name'] == cname, 'per_drop'] = round(count / l_data * 100, 2)
# get_zero_count

In [59]:
# # дропаем фичи, которые утеряны более чем на 10 %
# bad_list = list(get_zero_count[get_zero_count['per_drop'] > 10]['column_name'].values)
# bad_list.append('APPLICATION_MONTH')
# # сформируем лист колонок, которые можно восстановить
# not_so_bad_list = list(get_zero_count[get_zero_count['per_drop'] <= 10]['column_name'].values)

In [115]:
train = train[good_scores]

In [61]:
# count = 0
# for x in data_repare.isna().values.flatten():
#     if x:
#         count+=1
# count

In [62]:
# # define imputer
# imputer = KNNImputer(missing_values=np.nan, add_indicator=True)
# # fit on the dataset
# imputer.fit(data_repare[:10000])
# # transform the dataset
# features_trans = imputer.transform(data_repare[:10000])

In [63]:
# features_trans.to_csv(params.path + 'features_trans.csv', index=False)

In [64]:
# data_repare = data_repare.fillna(0).drop('incident', axis=1)

In [65]:
# data_repare['target'] = np.abs(model_df['PD'] - data_df['flg_90_12_add'])

In [116]:
test = data_repare[data_repare['target'].isna()][good_scores].reset_index(drop=True)
test

,Category_Feature_2,Feature_962,Feature_1083,Feature_960,Feature_69,Feature_1078,Feature_71,Feature_959,Feature_955,Feature_74,Feature_1085,Feature_73,Category_Feature_15,Feature_1042,Feature_1082,Feature_76,Feature_1834,Feature_1037,Feature_1833,Feature_1031,Feature_949,Feature_929,Feature_1072,Feature_1832,Feature_945,Feature_1044,Feature_167,Feature_191,Feature_111,Feature_1001,Feature_1052,Feature_159,Feature_70,Feature_175,Feature_72,Feature_127,Feature_1011,Feature_996,Category_Feature_3,Feature_1837,...,Feature_775,Feature_1736,Feature_68,Feature_1374,Feature_25,Feature_1270,Feature_1695,Feature_1333,Feature_1251,Feature_1250,Feature_1373,Feature_1292,Feature_1635,Feature_1586,Feature_1668,Feature_1709,Feature_1627,Feature_1676,Feature_104,Feature_152,Feature_694,Feature_710,Feature_120,Feature_678,Feature_1291,Feature_1332,Feature_297,Feature_830,Feature_798,Feature_814,Feature_313,Feature_281,Feature_265,Feature_691,Feature_133,Feature_675,Feature_117,Feature_149,Feature_707,Feature_101
0,0.611111,0.987380,1.000000,1.000000,0.079861,1.000000,0.283422,0.987420,1.000000,0.513393,0.990765,0.238739,1.0,1.000000,0.990794,-1.000000,0.241667,1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,0.981666,0.963397,-1.000000,-1.000000,-1.000000,1.0,1.000000,-1.000000,0.119565,-1.000000,0.040541,-1.000000,1.000000,1.0,0.2,-1.000000,...,-1.0,-1.000000,0.0,-1.0,0.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.203704,0.767093,0.854098,0.760644,0.059028,0.854098,0.048128,0.760644,0.760644,0.379464,0.852667,0.040541,1.0,0.561216,0.854098,0.102679,0.831481,0.560697,0.662963,0.292557,0.729535,0.729117,0.809454,0.159292,0.729377,0.561235,-1.000000,-1.000000,-1.000000,1.0,0.809160,-1.000000,0.103261,-1.000000,0.054054,-1.000000,0.292143,1.0,0.2,0.495327,...,-1.0,-1.000000,0.0,-1.0,0.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0.370370,0.891407,0.885303,0.887482,0.101389,0.855777,0.048128,0.852427,0.843792,0.651786,0.897723,0.040541,1.0,0.455880,0.876380,0.321429,0.422222,0.455236,0.277778,0.932203,0.971354,0.955717,0.967960,0.135272,0.961842,0.455903,-1.000000,-1.000000,-1.000000,1.0,0.956256,-1.000000,0.081522,-1.000000,0.040541,-1.000000,0.932164,1.0,0.2,0.214953,...,-1.0,-1.000000,0.0,-1.0,0.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0.148148,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.8,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,0.0,-1.000000,...,-1.0,-1.000000,0.0,-1.0,0.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,0.277778,0.821777,0.978903,0.972631,0.034028,0.978903,0.069519,0.819168,0.972631,0.218750,0.869117,0.058559,0.8,0.958057,0.826042,0.102679,1.000000,0.958008,0.662963,0.957627,0.972515,0.972472,0.978814,-1.000000,0.818296,0.523199,-1.000000,-1.000000,-1.000000,1.0,0.978781,-1.000000,0.076087,-1.000000,0.004505,-1.000000,0.957602,1.0,0.2,0.663551,...,-1.0,-1.000000,0.0,-1.0,0.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Обучение модели

In [117]:
X = train.reset_index(drop=True).values
y = data_repare['target'][~data_repare['target'].isna()].reset_index(drop=True).values.reshape(-1,1)
X_test = test.reset_index(drop=True).values

In [120]:
kf = KFold(n_splits=3, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X, X[test_index]
    y_train, y_valid = y, y[test_index]
    regressor = TabNetRegressor(**tabnet_params)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=2000,
              batch_size=1024, virtual_batch_size=128,
              num_workers=4,
              eval_metric=['mae'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(X_test)))

predictions = np.mean(predictions_array,axis=0)

Device used : cuda
epoch 0  | loss: 0.08927 | val_0_mae: 0.07772 |  0:00:05s
epoch 10 | loss: 0.03201 | val_0_mae: 0.0792  |  0:00:55s
epoch 20 | loss: 0.03148 | val_0_mae: 0.07498 |  0:01:45s
epoch 30 | loss: 0.03121 | val_0_mae: 0.08028 |  0:02:35s
epoch 40 | loss: 0.03097 | val_0_mae: 0.08444 |  0:03:25s
epoch 50 | loss: 0.03072 | val_0_mae: 0.09103 |  0:04:15s
epoch 60 | loss: 0.0305  | val_0_mae: 0.08432 |  0:05:05s
epoch 70 | loss: 0.03033 | val_0_mae: 0.0754  |  0:05:55s
epoch 80 | loss: 0.03004 | val_0_mae: 0.07803 |  0:06:45s
epoch 90 | loss: 0.02982 | val_0_mae: 0.0753  |  0:07:35s
epoch 100| loss: 0.0294  | val_0_mae: 0.07548 |  0:08:25s
epoch 110| loss: 0.02911 | val_0_mae: 0.07369 |  0:09:15s
epoch 120| loss: 0.02872 | val_0_mae: 0.07274 |  0:10:05s
epoch 130| loss: 0.0284  | val_0_mae: 0.07329 |  0:10:55s
epoch 140| loss: 0.02812 | val_0_mae: 0.07298 |  0:11:46s
epoch 150| loss: 0.02777 | val_0_mae: 0.0707  |  0:12:35s
epoch 160| loss: 0.02739 | val_0_mae: 0.07156 |  0:13

KeyboardInterrupt: ignored

Exception in thread Thread-5548:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/_utils/pin_memory.py", line 28, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.7/dist-packages/torch/multiprocessing/reductions.py", line 289, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.7/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.7/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/usr/lib/pyt

In [122]:
res = np.array([p[0] for p in predictions])

In [123]:
len(res)

1000

In [124]:
np.savetxt(conf.path + "result_v6.csv", res.astype('float64'), delimiter=',')

In [121]:
predictions = np.mean(predictions_array,axis=0)

In [110]:
# from sklearn.metrics import log_loss
# from pytorch_tabnet.metrics import Metric
# from sklearn.metrics import roc_auc_score, log_loss

# class LogitsLogLoss(Metric):
#     """
#     LogLoss with sigmoid applied
#     """

#     def __init__(self):
#         self._name = "logits_ll"
#         self._maximize = False

#     def __call__(self, y_true, y_pred):
#         """
#         Compute LogLoss of predictions.

#         Parameters
#         ----------
#         y_true: np.ndarray
#             Target matrix or vector
#         y_score: np.ndarray
#             Score matrix or vector

#         Returns
#         -------
#             float
#             LogLoss of predictions vs targets.
#         """
#         logits = 1 / (1 + np.exp(-y_pred))
#         aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
#         return np.mean(-aux)

In [111]:
# from iterstrat.ml_stratifiers import  MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
# from sklearn.metrics import roc_auc_score

In [112]:
# train_targets_scored = train['target']
# train = train.drop('target', axis=1)


# scores_auc_all= []
# test_cv_preds = []

# NB_SPLITS = 10
# mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
# oof_preds = []
# oof_targets = []
# scores = []
# scores_auc = []
# for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, train_targets_scored)):
#     print("FOLDS : ", fold_nb)

#     ## model
#     X_train, y_train = train.values[train_idx, :], train_targets_scored.values[train_idx, :]
#     X_val, y_val = train.values[val_idx, :], train_targets_scored.values[val_idx, :]
#     model = TabNetRegressor(**tabnet_params)

#     model.fit(X_train=X_train,
#               y_train=y_train,
#               eval_set=[(X_val, y_val)],
#               eval_name = ["val"],
#               eval_metric = ["logits_ll"],
#               max_epochs=MAX_EPOCH,
#               patience=20, batch_size=1024, virtual_batch_size=128,
#               num_workers=1, drop_last=False,
#               # use binary cross entropy as this is not a regression problem
#               loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)

#     preds_val = model.predict(X_val)
#     # Apply sigmoid to the predictions
#     preds =  1 / (1 + np.exp(-preds_val))
#     score = np.min(model.history["val_logits_ll"])
# #     name = cfg.save_name + f"_fold{fold_nb}"
# #     model.save_model(name)
#     ## save oof to compute the CV later
#     oof_preds.append(preds_val)
#     oof_targets.append(y_val)
#     scores.append(score)

#     # preds on test
#     preds_test = model.predict(X_test)
#     test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

# oof_preds_all = np.concatenate(oof_preds)
# oof_targets_all = np.concatenate(oof_targets)
# test_preds_all = np.stack(test_cv_preds)

In [113]:
# clf1_nopreproc = TabNetRegressor(cat_dims=cat_dims, cat_emb_dim=cat_emb_dim, cat_idxs=cat_idxs)

In [114]:
# clf1_nopreproc.fit(
#     x_train,y_train,
#     eval_set=[(x_train, y_train), (x_val, y_val)],
#     eval_name=['train', 'valid'],
#     eval_metric=['auc','accuracy'],
#     max_epochs=1000 , patience=50,
#     batch_size=256, virtual_batch_size=128,
#     num_workers=2,
#     weights=1,
#     drop_last=False
# )